## Task A

# Justification for referenced model

We use referencing as the N side of the query which is the fire data needs to be standalone for certain queries such as Query B.2 of the assignment.

Given:
The cardinality of the structure to be a one to many. One climate many fire data for any given date.
There will always be one climate data for a given date and there will be one or more than one fire data for the same date.
For example, in date 28/03/2017, it is not efficient to embed the fire data within the climate data for the given date as we add unnecessary load to the system to simply generate the climate data such as in query B.1 of the assignment which only requires the climate data.

Also, there is no requirement to update the entire object in either fire or climate data, ie, we don't have to change the date for which the specific fire occurred, which would cause the referencing of the fire data in climate to have 2 query updates. except for the precipitation which generates 0 or 99.99 which

Assumptions :
We are modelling our database with assuming that the type of queries provided in task 2 are most frequently used. 
We referenced the ID of hotspot data into climate data so that we have a list of unique object IDs of hotspot data in the climate data which can be queried with a look up if necessary.
Had we made the reference in the other way, it would have caused duplicate IDs of climate to occur in hotspot data, which we are not interested in.
Since there are no atomic updates required for the data as in in hotspot csv, we can assume that this model is efficient as our read to update ratios are very high (assuming that there may be very rare cases where we have to update the air temperature data).
By “Efficient Querying” we assume that the the results of the query accurately defines the requirements rather than the number of queries required to access the business requirements.
For example, if we had used an embedded model, we would have obtained unnecessary data each time the query for climate data was performed where it would also retrieve all the fire data on that day.
We have made use of indexing based on the date key on both the collections to avoid overhead due to collection scans and for optimize querying.


# Example of our model

## Climate Collection

{'Air_Temperature_Celcius': 14,
  'Date': '2017-04-11',
  'Hotspot_ID': [2066,
                 2067,
                 2068,
                 2069,
                 2070,
                 2071,
                 2072,
                 2073,
                 2074,
                 2075,
                 2076,
                 2077,
                 2078,
                 2079,
                 2080,
                 2081,
                 2082,
                 2083,
                 2084,
                 2085,
                 2086,
                 2087,
                 2088,
                 2089],
  'Max_Wind_Speed': 13.0,
  'Precipitation': ' 0.00G',
  'Relative_Humidity': 52.5,
  'Station': 948701,
  'Windspeed_Knots': 7.0,
  '_id': 100}
  
  
  ## Hotspot Collection
  {'Confidence': 82,
  'Date': '2017-12-27',
  'Datetime': Timestamp('2017-12-27 00:02:15'),
  'Latitude': -35.541,
  'Longitude': 143.311,
  'Surface_Temperature_Celcius': 63,
  '_id': 1}

## Task B
### Importing Libraries

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import numpy as np

### Connecting to MondoDB instance and fetching database

In [2]:
#Setting up Mongo local instnce client
client = MongoClient()
db = client.fit5148_assignment_db
#client = MongoClient('localhost', 27017)


In [3]:
db.climate_historic.drop()
db.hotspot_historic.drop()


### Defining field names for each CSV File.

In [4]:
# Column names for each collection.
fieldNameClimate = ["Station", "Date", "Air_Temperature_Celcius" ,"Relative_Humidity", "Windspeed_Knots", "Max_Wind_Speed",
                    "Precipitation"]
fieldNameHotspot = ["Latitude", "Longitude", "Datetime", "Confidence", "Date", "Surface_Temperature_Celcius"]

### Reading CSV file data into DataFrames.

In [5]:
# Making a dataframe with column names as specified above.
climateData = pd.read_csv("climate_historic.csv", sep=",", names=fieldNameClimate, header=0)
# Converting date fields to proper date time fields from specified format.
climateData['Date'] = pd.to_datetime(climateData['Date'], format="%d/%m/%Y")
# Converting date time field to string type so that json doesn't convert it back to unusable timestamp.
climateData['Date'] = climateData['Date'].dt.strftime('%Y-%m-%d')
# Creating indexes for storing into DB as _id
climateData = climateData.reset_index()
climateData.rename(columns={'index':'_id'}, inplace=True)
climateData['Hotspot_ID'] = None
climateData.head()

,_id,Station,Date,Air_Temperature_Celcius,Relative_Humidity,Windspeed_Knots,Max_Wind_Speed,Precipitation,Hotspot_ID
0,0,948700,2016-12-31,19,56.8,7.9,11.1,0.00I,None
1,1,948700,2017-01-02,15,50.7,9.2,13.0,0.02G,None
2,2,948700,2017-01-03,16,53.6,8.1,15.0,0.00G,None
3,3,948700,2017-01-04,24,61.6,7.7,14.0,0.00I,None
4,4,948700,2017-01-05,24,62.3,7.0,13.0,0.00I,None


In [6]:
# Making a dataframe with column names as specified above.
hotspotData = pd.read_csv("hotspot_historic.csv", sep=",", names=fieldNameHotspot, header=0)
# Converting date fields to proper date time fields from specified format.
hotspotData['Datetime'] = pd.to_datetime(hotspotData['Datetime'])
hotspotData['Date'] = pd.to_datetime(hotspotData['Date'], format="%d/%m/%Y")
# Converting date time field to string type so that json doesn't convert it back to unusable timestamp.
hotspotData['Date'] = hotspotData['Date'].dt.strftime('%Y-%m-%d')
# Creating indexes for storing into DB as _id
hotspotData = hotspotData.reset_index()
hotspotData.rename(columns={'index':'_id'}, inplace=True)
hotspotData.head()

,_id,Latitude,Longitude,Datetime,Confidence,Date,Surface_Temperature_Celcius
0,0,-37.966,145.051,2017-12-27 04:16:51,78,2017-12-27,68
1,1,-35.541,143.311,2017-12-27 00:02:15,82,2017-12-27,63
2,2,-35.554,143.307,2017-12-27 00:02:15,67,2017-12-27,53
3,3,-35.543,143.316,2017-12-27 00:02:14,86,2017-12-27,67
4,4,-37.708,145.100,2017-12-25 04:29:08,80,2017-12-25,54


In [7]:
for index, row in climateData.iterrows():
    date = row['Date']
    # Match the hotspot data values with climate data by dates
    hotspotDates = hotspotData[hotspotData['Date'] == date]
    indexes = hotspotDates['_id'].values.tolist()
    # Access a single value using a label for a series.
    climateData.at[index, 'Hotspot_ID'] = indexes
climateData.head(100)

,_id,Station,Date,Air_Temperature_Celcius,Relative_Humidity,Windspeed_Knots,Max_Wind_Speed,Precipitation,Hotspot_ID
0,0,948700,2016-12-31,19,56.8,7.9,11.1,0.00I,[]
1,1,948700,2017-01-02,15,50.7,9.2,13.0,0.02G,[]
2,2,948700,2017-01-03,16,53.6,8.1,15.0,0.00G,[]
3,3,948700,2017-01-04,24,61.6,7.7,14.0,0.00I,[]
4,4,948700,2017-01-05,24,62.3,7.0,13.0,0.00I,[]
5,5,948700,2017-01-06,26,60.8,6.9,12.0,0.00I,[]
6,6,948700,2017-01-07,32,54.1,12.8,19.0,0.00I,[]
7,7,948700,2017-01-08,20,57.7,9.9,18.1,0.00I,[]
8,8,948700,2017-01-09,19,56.3,5.8,8.9,0.00I,[]
9,9,948700,2017-01-10,20,57.0,8.7,13.0,0.00I,[]


### Converting Dataframe to JSON objects (Dictionary).

In [8]:
# Converting Dataframe to records orient json object.
climateDict = climateData.to_dict(orient="records")
pprint(climateDict)

[{'Air_Temperature_Celcius': 19,
  'Date': '2016-12-31',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 11.1,
  'Precipitation': ' 0.00I',
  'Relative_Humidity': 56.8,
  'Station': 948700,
  'Windspeed_Knots': 7.9,
  '_id': 0},
 {'Air_Temperature_Celcius': 15,
  'Date': '2017-01-02',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 13.0,
  'Precipitation': ' 0.02G',
  'Relative_Humidity': 50.7,
  'Station': 948700,
  'Windspeed_Knots': 9.2,
  '_id': 1},
 {'Air_Temperature_Celcius': 16,
  'Date': '2017-01-03',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 15.0,
  'Precipitation': ' 0.00G',
  'Relative_Humidity': 53.6,
  'Station': 948700,
  'Windspeed_Knots': 8.1,
  '_id': 2},
 {'Air_Temperature_Celcius': 24,
  'Date': '2017-01-04',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 14.0,
  'Precipitation': ' 0.00I',
  'Relative_Humidity': 61.6,
  'Station': 948700,
  'Windspeed_Knots': 7.7,
  '_id': 3},
 {'Air_Temperature_Celcius': 24,
  'Date': '2017-01-05',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 13.0,
  'Precipitat

  'Station': 948701,
  'Windspeed_Knots': 8.1,
  '_id': 75},
 {'Air_Temperature_Celcius': 21,
  'Date': '2017-03-18',
  'Hotspot_ID': [2619, 2620, 2621],
  'Max_Wind_Speed': 11.1,
  'Precipitation': ' 0.00I',
  'Relative_Humidity': 52.8,
  'Station': 948701,
  'Windspeed_Knots': 4.1,
  '_id': 76},
 {'Air_Temperature_Celcius': 24,
  'Date': '2017-03-19',
  'Hotspot_ID': [2598,
                 2599,
                 2600,
                 2601,
                 2602,
                 2603,
                 2604,
                 2605,
                 2606,
                 2607,
                 2608,
                 2609,
                 2610,
                 2611,
                 2612,
                 2613,
                 2614,
                 2615,
                 2616,
                 2617,
                 2618],
  'Max_Wind_Speed': 11.1,
  'Precipitation': ' 0.00I',
  'Relative_Humidity': 55.2,
  'Station': 948701,
  'Windspeed_Knots': 5.8,
  '_id': 77},
 {'Air_Temperat

                 2078,
                 2079,
                 2080,
                 2081,
                 2082,
                 2083,
                 2084,
                 2085,
                 2086,
                 2087,
                 2088,
                 2089],
  'Max_Wind_Speed': 13.0,
  'Precipitation': ' 0.00G',
  'Relative_Humidity': 52.5,
  'Station': 948701,
  'Windspeed_Knots': 7.0,
  '_id': 100},
 {'Air_Temperature_Celcius': 14,
  'Date': '2017-04-12',
  'Hotspot_ID': [1997,
                 1998,
                 1999,
                 2000,
                 2001,
                 2002,
                 2003,
                 2004,
                 2005,
                 2006,
                 2007,
                 2008,
                 2009,
                 2010,
                 2011,
                 2012,
                 2013,
                 2014,
                 2015,
                 2016,
                 2017,
                 2018,
              

                 1454,
                 1455,
                 1456,
                 1457,
                 1458,
                 1459,
                 1460,
                 1461,
                 1462,
                 1463,
                 1464,
                 1465,
                 1466,
                 1467,
                 1468,
                 1469,
                 1470,
                 1471,
                 1472,
                 1473,
                 1474,
                 1475,
                 1476,
                 1477,
                 1478,
                 1479,
                 1480,
                 1481,
                 1482,
                 1483,
                 1484,
                 1485,
                 1486,
                 1487,
                 1488,
                 1489,
                 1490,
                 1491,
                 1492,
                 1493,
                 1494,
                 1495,
                 1496],
  'Max_Win

  'Precipitation': ' 0.00I',
  'Relative_Humidity': 48.7,
  'Station': 948701,
  'Windspeed_Knots': 10.5,
  '_id': 141},
 {'Air_Temperature_Celcius': 15,
  'Date': '2017-05-23',
  'Hotspot_ID': [389, 390, 391, 392, 393],
  'Max_Wind_Speed': 13.0,
  'Precipitation': ' 0.01G',
  'Relative_Humidity': 49.9,
  'Station': 948701,
  'Windspeed_Knots': 7.9,
  '_id': 142},
 {'Air_Temperature_Celcius': 14,
  'Date': '2017-05-24',
  'Hotspot_ID': [386, 387, 388],
  'Max_Wind_Speed': 13.0,
  'Precipitation': ' 0.00G',
  'Relative_Humidity': 48.4,
  'Station': 948701,
  'Windspeed_Knots': 7.5,
  '_id': 143},
 {'Air_Temperature_Celcius': 14,
  'Date': '2017-05-25',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 13.0,
  'Precipitation': ' 0.12G',
  'Relative_Humidity': 50.7,
  'Station': 948701,
  'Windspeed_Knots': 8.6,
  '_id': 144},
 {'Air_Temperature_Celcius': 13,
  'Date': '2017-05-26',
  'Hotspot_ID': [382, 383, 384, 385],
  'Max_Wind_Speed': 15.9,
  'Precipitation': ' 0.00G',
  'Relative_Humidity': 4

  'Date': '2017-07-17',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 21.0,
  'Precipitation': ' 0.00I',
  'Relative_Humidity': 41.8,
  'Station': 948701,
  'Windspeed_Knots': 14.7,
  '_id': 197},
 {'Air_Temperature_Celcius': 10,
  'Date': '2017-07-18',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 14.0,
  'Precipitation': ' 0.39G',
  'Relative_Humidity': 44.8,
  'Station': 948701,
  'Windspeed_Knots': 8.8,
  '_id': 198},
 {'Air_Temperature_Celcius': 10,
  'Date': '2017-07-19',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 16.9,
  'Precipitation': ' 0.04G',
  'Relative_Humidity': 43.5,
  'Station': 948701,
  'Windspeed_Knots': 12.0,
  '_id': 199},
 {'Air_Temperature_Celcius': 7,
  'Date': '2017-07-20',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 15.0,
  'Precipitation': ' 0.12G',
  'Relative_Humidity': 40.5,
  'Station': 948701,
  'Windspeed_Knots': 8.1,
  '_id': 200},
 {'Air_Temperature_Celcius': 8,
  'Date': '2017-07-21',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 14.0,
  'Precipitation': ' 0.00G',
  'Relati

                 230,
                 231,
                 232,
                 233,
                 234,
                 235,
                 236,
                 237,
                 238],
  'Max_Wind_Speed': 23.9,
  'Precipitation': ' 0.16G',
  'Relative_Humidity': 44.0,
  'Station': 948702,
  'Windspeed_Knots': 14.4,
  '_id': 266},
 {'Air_Temperature_Celcius': 9,
  'Date': '2017-09-25',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 12.0,
  'Precipitation': ' 0.01G',
  'Relative_Humidity': 42.1,
  'Station': 948702,
  'Windspeed_Knots': 7.6,
  '_id': 267},
 {'Air_Temperature_Celcius': 11,
  'Date': '2017-09-26',
  'Hotspot_ID': [210],
  'Max_Wind_Speed': 8.9,
  'Precipitation': ' 0.00G',
  'Relative_Humidity': 40.8,
  'Station': 948702,
  'Windspeed_Knots': 6.4,
  '_id': 268},
 {'Air_Temperature_Celcius': 15,
  'Date': '2017-09-27',
  'Hotspot_ID': [203, 204, 205, 206, 207, 208, 209],
  'Max_Wind_Speed': 19.0,
  'Precipitation': ' 0.00I',
  'Relative_Humidity': 44.8,
  'Station': 

  '_id': 332},
 {'Air_Temperature_Celcius': 26,
  'Date': '2017-11-30',
  'Hotspot_ID': [40,
                 41,
                 42,
                 43,
                 44,
                 45,
                 46,
                 47,
                 48,
                 49,
                 50,
                 51,
                 52,
                 53,
                 54,
                 55,
                 56,
                 57,
                 58,
                 59,
                 60,
                 61,
                 62,
                 63,
                 64,
                 65,
                 66,
                 67,
                 68,
                 69,
                 70],
  'Max_Wind_Speed': 15.0,
  'Precipitation': ' 0.20G',
  'Relative_Humidity': 61.0,
  'Station': 948702,
  'Windspeed_Knots': 9.3,
  '_id': 333},
 {'Air_Temperature_Celcius': 17,
  'Date': '2017-12-01',
  'Hotspot_ID': [],
  'Max_Wind_Speed': 15.0,
  'Precipitation': ' 0.75G'

In [9]:
# Converting Dataframe to records orient json object.
hotspotDict = hotspotData.to_dict(orient="records")
pprint(hotspotDict)

[{'Confidence': 78,
  'Date': '2017-12-27',
  'Datetime': Timestamp('2017-12-27 04:16:51'),
  'Latitude': -37.966,
  'Longitude': 145.05100000000002,
  'Surface_Temperature_Celcius': 68,
  '_id': 0},
 {'Confidence': 82,
  'Date': '2017-12-27',
  'Datetime': Timestamp('2017-12-27 00:02:15'),
  'Latitude': -35.541,
  'Longitude': 143.311,
  'Surface_Temperature_Celcius': 63,
  '_id': 1},
 {'Confidence': 67,
  'Date': '2017-12-27',
  'Datetime': Timestamp('2017-12-27 00:02:15'),
  'Latitude': -35.554,
  'Longitude': 143.30700000000002,
  'Surface_Temperature_Celcius': 53,
  '_id': 2},
 {'Confidence': 86,
  'Date': '2017-12-27',
  'Datetime': Timestamp('2017-12-27 00:02:14'),
  'Latitude': -35.543,
  'Longitude': 143.316,
  'Surface_Temperature_Celcius': 67,
  '_id': 3},
 {'Confidence': 80,
  'Date': '2017-12-25',
  'Datetime': Timestamp('2017-12-25 04:29:08'),
  'Latitude': -37.708,
  'Longitude': 145.1,
  'Surface_Temperature_Celcius': 54,
  '_id': 4},
 {'Confidence': 65,
  'Date': '2017

  'Surface_Temperature_Celcius': 84,
  '_id': 78},
 {'Confidence': 50,
  'Date': '2017-11-28',
  'Datetime': Timestamp('2017-11-28 00:33:14'),
  'Latitude': -37.749,
  'Longitude': 148.297,
  'Surface_Temperature_Celcius': 42,
  '_id': 79},
 {'Confidence': 84,
  'Date': '2017-11-23',
  'Datetime': Timestamp('2017-11-23 04:28:59'),
  'Latitude': -37.946,
  'Longitude': 144.342,
  'Surface_Temperature_Celcius': 60,
  '_id': 80},
 {'Confidence': 93,
  'Date': '2017-11-23',
  'Datetime': Timestamp('2017-11-23 04:28:59'),
  'Latitude': -37.945,
  'Longitude': 144.35399999999998,
  'Surface_Temperature_Celcius': 73,
  '_id': 81},
 {'Confidence': 68,
  'Date': '2017-11-23',
  'Datetime': Timestamp('2017-11-23 00:15:12'),
  'Latitude': -38.141,
  'Longitude': 143.183,
  'Surface_Temperature_Celcius': 46,
  '_id': 82},
 {'Confidence': 63,
  'Date': '2017-11-23',
  'Datetime': Timestamp('2017-11-23 00:14:47'),
  'Latitude': -36.952,
  'Longitude': 144.972,
  'Surface_Temperature_Celcius': 60,
  

  'Longitude': 145.607,
  'Surface_Temperature_Celcius': 51,
  '_id': 147},
 {'Confidence': 91,
  'Date': '2017-10-17',
  'Datetime': Timestamp('2017-10-17 04:10:45'),
  'Latitude': -37.247,
  'Longitude': 141.278,
  'Surface_Temperature_Celcius': 68,
  '_id': 148},
 {'Confidence': 58,
  'Date': '2017-10-17',
  'Datetime': Timestamp('2017-10-17 04:10:38'),
  'Latitude': -37.586999999999996,
  'Longitude': 142.47899999999998,
  'Surface_Temperature_Celcius': 41,
  '_id': 149},
 {'Confidence': 82,
  'Date': '2017-10-17',
  'Datetime': Timestamp('2017-10-17 04:10:32'),
  'Latitude': -37.805,
  'Longitude': 144.15,
  'Surface_Temperature_Celcius': 55,
  '_id': 150},
 {'Confidence': 62,
  'Date': '2017-10-16',
  'Datetime': Timestamp('2017-10-16 23:56:43'),
  'Latitude': -37.288000000000004,
  'Longitude': 144.39,
  'Surface_Temperature_Celcius': 36,
  '_id': 151},
 {'Confidence': 100,
  'Date': '2017-10-15',
  'Datetime': Timestamp('2017-10-15 04:23:01'),
  'Latitude': -37.294000000000004,

  'Longitude': 148.06,
  'Surface_Temperature_Celcius': 48,
  '_id': 226},
 {'Confidence': 84,
  'Date': '2017-09-24',
  'Datetime': Timestamp('2017-09-24 13:30:07'),
  'Latitude': -37.438,
  'Longitude': 148.09,
  'Surface_Temperature_Celcius': 50,
  '_id': 227},
 {'Confidence': 100,
  'Date': '2017-09-24',
  'Datetime': Timestamp('2017-09-24 04:04:19'),
  'Latitude': -37.406,
  'Longitude': 148.123,
  'Surface_Temperature_Celcius': 88,
  '_id': 228},
 {'Confidence': 58,
  'Date': '2017-09-24',
  'Datetime': Timestamp('2017-09-24 04:04:19'),
  'Latitude': -37.397,
  'Longitude': 148.121,
  'Surface_Temperature_Celcius': 58,
  '_id': 229},
 {'Confidence': 68,
  'Date': '2017-09-24',
  'Datetime': Timestamp('2017-09-24 04:04:18'),
  'Latitude': -37.452,
  'Longitude': 148.115,
  'Surface_Temperature_Celcius': 77,
  '_id': 230},
 {'Confidence': 68,
  'Date': '2017-09-24',
  'Datetime': Timestamp('2017-09-24 04:04:18'),
  'Latitude': -37.465,
  'Longitude': 148.153,
  'Surface_Temperature

  'Surface_Temperature_Celcius': 50,
  '_id': 296},
 {'Confidence': 71,
  'Date': '2017-07-29',
  'Datetime': Timestamp('2017-07-29 04:07:50'),
  'Latitude': -37.8565,
  'Longitude': 147.2532,
  'Surface_Temperature_Celcius': 46,
  '_id': 297},
 {'Confidence': 63,
  'Date': '2017-07-14',
  'Datetime': Timestamp('2017-07-14 04:58:10'),
  'Latitude': -38.5194,
  'Longitude': 143.4522,
  'Surface_Temperature_Celcius': 42,
  '_id': 298},
 {'Confidence': 61,
  'Date': '2017-07-14',
  'Datetime': Timestamp('2017-07-14 04:50:20'),
  'Latitude': -38.5233,
  'Longitude': 143.433,
  'Surface_Temperature_Celcius': 41,
  '_id': 299},
 {'Confidence': 83,
  'Date': '2017-07-06',
  'Datetime': Timestamp('2017-07-06 04:08:00'),
  'Latitude': -37.8662,
  'Longitude': 143.4082,
  'Surface_Temperature_Celcius': 56,
  '_id': 300},
 {'Confidence': 91,
  'Date': '2017-07-06',
  'Datetime': Timestamp('2017-07-06 04:02:00'),
  'Latitude': -37.8644,
  'Longitude': 143.4263,
  'Surface_Temperature_Celcius': 69,

...

  'Latitude': -37.6387,
  'Longitude': 142.9032,
  'Surface_Temperature_Celcius': 88,
  '_id': 2283},
 {'Confidence': 77,
  'Date': '2017-04-05',
  'Datetime': Timestamp('2017-04-05 05:15:50'),
  'Latitude': -38.0045,
  'Longitude': 143.9075,
  'Surface_Temperature_Celcius': 64,
  '_id': 2284},
 {'Confidence': 66,
  'Date': '2017-04-05',
  'Datetime': Timestamp('2017-04-05 05:15:40'),
  'Latitude': -37.7522,
  'Longitude': 142.3183,
  'Surface_Temperature_Celcius': 50,
  '_id': 2285},
 {'Confidence': 66,
  'Date': '2017-04-05',
  'Datetime': Timestamp('2017-04-05 05:15:30'),
  'Latitude': -36.9396,
  'Longitude': 141.9303,
  'Surface_Temperature_Celcius': 45,
  '_id': 2286},
 {'Confidence': 93,
  'Date': '2017-04-05',
  'Datetime': Timestamp('2017-04-05 05:15:30'),
  'Latitude': -37.6439,
  'Longitude': 142.91299999999998,
  'Surface_Temperature_Celcius': 83,
  '_id': 2287},
 {'Confidence': 65,
  'Date': '2017-04-05',
  'Datetime': Timestamp('2017-04-05 03:45:10'),
  'Latitude': -37.78

  'Surface_Temperature_Celcius': 50,
  '_id': 2332},
 {'Confidence': 100,
  'Date': '2017-04-04',
  'Datetime': Timestamp('2017-04-04 04:39:20'),
  'Latitude': -36.1849,
  'Longitude': 145.9109,
  'Surface_Temperature_Celcius': 107,
  '_id': 2333},
 {'Confidence': 74,
  'Date': '2017-04-04',
  'Datetime': Timestamp('2017-04-04 04:39:20'),
  'Latitude': -37.2109,
  'Longitude': 142.7966,
  'Surface_Temperature_Celcius': 52,
  '_id': 2334},
 {'Confidence': 50,
  'Date': '2017-04-04',
  'Datetime': Timestamp('2017-04-04 04:38:10'),
  'Latitude': -36.6951,
  'Longitude': 145.424,
  'Surface_Temperature_Celcius': 43,
  '_id': 2335},
 {'Confidence': 76,
  'Date': '2017-04-04',
  'Datetime': Timestamp('2017-04-04 04:37:30'),
  'Latitude': -36.1376,
  'Longitude': 145.84,
  'Surface_Temperature_Celcius': 55,
  '_id': 2336},
 {'Confidence': 88,
  'Date': '2017-04-04',
  'Datetime': Timestamp('2017-04-04 04:37:10'),
  'Latitude': -36.6965,
  'Longitude': 142.7116,
  'Surface_Temperature_Celcius'

 {'Confidence': 87,
  'Date': '2017-04-03',
  'Datetime': Timestamp('2017-04-03 03:59:10'),
  'Latitude': -37.3735,
  'Longitude': 145.886,
  'Surface_Temperature_Celcius': 96,
  '_id': 2411},
 {'Confidence': 75,
  'Date': '2017-04-03',
  'Datetime': Timestamp('2017-04-03 03:58:20'),
  'Latitude': -37.5422,
  'Longitude': 143.2276,
  'Surface_Temperature_Celcius': 49,
  '_id': 2412},
 {'Confidence': 93,
  'Date': '2017-04-03',
  'Datetime': Timestamp('2017-04-03 03:57:50'),
  'Latitude': -37.7465,
  'Longitude': 143.3806,
  'Surface_Temperature_Celcius': 72,
  '_id': 2413},
 {'Confidence': 77,
  'Date': '2017-04-03',
  'Datetime': Timestamp('2017-04-03 03:56:40'),
  'Latitude': -37.3094,
  'Longitude': 146.1945,
  'Surface_Temperature_Celcius': 60,
  '_id': 2414},
 {'Confidence': 82,
  'Date': '2017-04-03',
  'Datetime': Timestamp('2017-04-03 03:56:20'),
  'Latitude': -36.7199,
  'Longitude': 141.3946,
  'Surface_Temperature_Celcius': 55,
  '_id': 2415},
 {'Confidence': 79,
  'Date': '

 {'Confidence': 69,
  'Date': '2017-03-29',
  'Datetime': Timestamp('2017-03-29 00:48:40'),
  'Latitude': -34.2648,
  'Longitude': 141.6325,
  'Surface_Temperature_Celcius': 51,
  '_id': 2511},
 {'Confidence': 88,
  'Date': '2017-03-28',
  'Datetime': Timestamp('2017-03-28 04:38:00'),
  'Latitude': -36.4721,
  'Longitude': 142.2586,
  'Surface_Temperature_Celcius': 64,
  '_id': 2512},
 {'Confidence': 72,
  'Date': '2017-03-28',
  'Datetime': Timestamp('2017-03-28 04:37:30'),
  'Latitude': -36.7241,
  'Longitude': 141.0073,
  'Surface_Temperature_Celcius': 46,
  '_id': 2513},
 {'Confidence': 86,
  'Date': '2017-03-28',
  'Datetime': Timestamp('2017-03-28 04:33:40'),
  'Latitude': -38.004,
  'Longitude': 143.9249,
  'Surface_Temperature_Celcius': 60,
  '_id': 2514},
 {'Confidence': 92,
  'Date': '2017-03-28',
  'Datetime': Timestamp('2017-03-28 04:33:30'),
  'Latitude': -36.4557,
  'Longitude': 142.2434,
  'Surface_Temperature_Celcius': 71,
  '_id': 2515},
 {'Confidence': 95,
  'Date': '

 {'Confidence': 84,
  'Date': '2017-03-19',
  'Datetime': Timestamp('2017-03-19 04:32:50'),
  'Latitude': -37.671,
  'Longitude': 144.6536,
  'Surface_Temperature_Celcius': 58,
  '_id': 2611},
 {'Confidence': 66,
  'Date': '2017-03-19',
  'Datetime': Timestamp('2017-03-19 04:32:50'),
  'Latitude': -37.9277,
  'Longitude': 143.477,
  'Surface_Temperature_Celcius': 55,
  '_id': 2612},
 {'Confidence': 100,
  'Date': '2017-03-19',
  'Datetime': Timestamp('2017-03-19 04:32:40'),
  'Latitude': -37.4224,
  'Longitude': 147.0511,
  'Surface_Temperature_Celcius': 113,
  '_id': 2613},
 {'Confidence': 92,
  'Date': '2017-03-19',
  'Datetime': Timestamp('2017-03-19 04:32:40'),
  'Latitude': -37.6625,
  'Longitude': 144.287,
  'Surface_Temperature_Celcius': 70,
  '_id': 2614},
 {'Confidence': 100,
  'Date': '2017-03-19',
  'Datetime': Timestamp('2017-03-19 04:32:40'),
  'Latitude': -37.4375,
  'Longitude': 147.0336,
  'Surface_Temperature_Celcius': 106,
  '_id': 2615},
 {'Confidence': 73,
  'Date':

### Inserting JSON objects to respective collections.

In [10]:
db.climate_historic.insert_many(climateDict)

In [11]:
db.hotspot_historic.insert_many(hotspotDict)

In [12]:
# print("Number of documents in climate_Histroic collection:",db.climate_historic.count_documents({}))
# print("Number of documents in hostpsot_Histroic collection:",db.hotspot_historic.count_documents({}))

### Queries

#### 1. Find climate data on ​10th December 2017

In [13]:
query = {"Date" : "2017-12-10"}
results = db.climate_historic.find(query)
for result in results:
    pprint(result)

{'Air_Temperature_Celcius': 17,
 'Date': '2017-12-10',
 'Hotspot_ID': [29, 30],
 'Max_Wind_Speed': 14.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 53.5,
 'Station': 948702,
 'Windspeed_Knots': 7.3,
 '_id': 343}


#### 2. Find the latitude, longitude, surface temperature ( °C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C

In [14]:
results = db.hotspot_historic.find(
{
    "Surface_Temperature_Celcius": {
        "$gte": 60,
        "$lte": 100
    }
},
{
    "Latitude" : 1,
    "Longitude" : 1,
    "Surface_Temperature_Celcius" : 1,
    "Confidence":1 
})
    
for result in results:
    pprint(result)

{'Confidence': 78,
 'Latitude': -37.966,
 'Longitude': 145.05100000000002,
 'Surface_Temperature_Celcius': 68,
 '_id': 0}
{'Confidence': 82,
 'Latitude': -35.541,
 'Longitude': 143.311,
 'Surface_Temperature_Celcius': 63,
 '_id': 1}
{'Confidence': 86,
 'Latitude': -35.543,
 'Longitude': 143.316,
 'Surface_Temperature_Celcius': 67,
 '_id': 3}
{'Confidence': 93,
 'Latitude': -37.875,
 'Longitude': 142.51,
 'Surface_Temperature_Celcius': 73,
 '_id': 10}
{'Confidence': 95,
 'Latitude': -37.613,
 'Longitude': 149.305,
 'Surface_Temperature_Celcius': 75,
 '_id': 12}
{'Confidence': 90,
 'Latitude': -37.624,
 'Longitude': 149.314,
 'Surface_Temperature_Celcius': 66,
 '_id': 14}
{'Confidence': 79,
 'Latitude': -37.649,
 'Longitude': 149.322,
 'Surface_Temperature_Celcius': 60,
 '_id': 16}
{'Confidence': 93,
 'Latitude': -38.056999999999995,
 'Longitude': 144.211,
 'Surface_Temperature_Celcius': 73,
 '_id': 17}
{'Confidence': 82,
 'Latitude': -37.650999999999996,
 'Longitude': 149.345,
 'Surface

 'Latitude': -37.2512,
 'Longitude': 144.3799,
 'Surface_Temperature_Celcius': 60,
 '_id': 456}
{'Confidence': 83,
 'Latitude': -38.0395,
 'Longitude': 146.0486,
 'Surface_Temperature_Celcius': 63,
 '_id': 457}
{'Confidence': 100,
 'Latitude': -36.3433,
 'Longitude': 143.4369,
 'Surface_Temperature_Celcius': 94,
 '_id': 468}
{'Confidence': 87,
 'Latitude': -37.0167,
 'Longitude': 143.8586,
 'Surface_Temperature_Celcius': 62,
 '_id': 471}
{'Confidence': 100,
 'Latitude': -36.345,
 'Longitude': 143.4256,
 'Surface_Temperature_Celcius': 87,
 '_id': 472}
{'Confidence': 90,
 'Latitude': -36.7032,
 'Longitude': 144.0992,
 'Surface_Temperature_Celcius': 66,
 '_id': 473}
{'Confidence': 96,
 'Latitude': -36.3402,
 'Longitude': 141.7429,
 'Surface_Temperature_Celcius': 77,
 '_id': 488}
{'Confidence': 89,
 'Latitude': -36.5661,
 'Longitude': 142.2956,
 'Surface_Temperature_Celcius': 65,
 '_id': 493}
{'Confidence': 85,
 'Latitude': -36.5023,
 'Longitude': 143.6038,
 'Surface_Temperature_Celcius': 

{'Confidence': 100,
 'Latitude': -36.7742,
 'Longitude': 145.1991,
 'Surface_Temperature_Celcius': 89,
 '_id': 842}
{'Confidence': 87,
 'Latitude': -36.4379,
 'Longitude': 141.3392,
 'Surface_Temperature_Celcius': 63,
 '_id': 847}
{'Confidence': 91,
 'Latitude': -36.5061,
 'Longitude': 144.5675,
 'Surface_Temperature_Celcius': 68,
 '_id': 851}
{'Confidence': 100,
 'Latitude': -36.4035,
 'Longitude': 140.9935,
 'Surface_Temperature_Celcius': 95,
 '_id': 855}
{'Confidence': 99,
 'Latitude': -36.4286,
 'Longitude': 141.1567,
 'Surface_Temperature_Celcius': 84,
 '_id': 864}
{'Confidence': 87,
 'Latitude': -36.4402,
 'Longitude': 141.3259,
 'Surface_Temperature_Celcius': 61,
 '_id': 865}
{'Confidence': 91,
 'Latitude': -36.3635,
 'Longitude': 142.2809,
 'Surface_Temperature_Celcius': 68,
 '_id': 867}
{'Confidence': 100,
 'Latitude': -36.664,
 'Longitude': 142.8802,
 'Surface_Temperature_Celcius': 87,
 '_id': 868}
{'Confidence': 91,
 'Latitude': -36.6564,
 'Longitude': 142.8615,
 'Surface_Te

 '_id': 1185}
{'Confidence': 93,
 'Latitude': -36.5871,
 'Longitude': 144.4958,
 'Surface_Temperature_Celcius': 72,
 '_id': 1193}
{'Confidence': 89,
 'Latitude': -36.2618,
 'Longitude': 141.8783,
 'Surface_Temperature_Celcius': 65,
 '_id': 1196}
{'Confidence': 90,
 'Latitude': -36.6828,
 'Longitude': 144.784,
 'Surface_Temperature_Celcius': 66,
 '_id': 1198}
{'Confidence': 89,
 'Latitude': -37.4843,
 'Longitude': 143.0592,
 'Surface_Temperature_Celcius': 66,
 '_id': 1201}
{'Confidence': 88,
 'Latitude': -37.1262,
 'Longitude': 141.7213,
 'Surface_Temperature_Celcius': 63,
 '_id': 1213}
{'Confidence': 88,
 'Latitude': -36.2851,
 'Longitude': 145.8,
 'Surface_Temperature_Celcius': 64,
 '_id': 1223}
{'Confidence': 88,
 'Latitude': -36.5577,
 'Longitude': 145.3929,
 'Surface_Temperature_Celcius': 64,
 '_id': 1224}
{'Confidence': 86,
 'Latitude': -37.7751,
 'Longitude': 143.0494,
 'Surface_Temperature_Celcius': 61,
 '_id': 1225}
{'Confidence': 94,
 'Latitude': -37.7274,
 'Longitude': 142.95

 '_id': 1627}
{'Confidence': 84,
 'Latitude': -36.7432,
 'Longitude': 141.3296,
 'Surface_Temperature_Celcius': 79,
 '_id': 1629}
{'Confidence': 84,
 'Latitude': -36.7531,
 'Longitude': 141.3253,
 'Surface_Temperature_Celcius': 78,
 '_id': 1631}
{'Confidence': 76,
 'Latitude': -36.7394,
 'Longitude': 141.3203,
 'Surface_Temperature_Celcius': 63,
 '_id': 1632}
{'Confidence': 86,
 'Latitude': -36.0374,
 'Longitude': 143.8136,
 'Surface_Temperature_Celcius': 61,
 '_id': 1634}
{'Confidence': 99,
 'Latitude': -36.7943,
 'Longitude': 143.9571,
 'Surface_Temperature_Celcius': 84,
 '_id': 1635}
{'Confidence': 87,
 'Latitude': -36.0375,
 'Longitude': 143.8172,
 'Surface_Temperature_Celcius': 63,
 '_id': 1636}
{'Confidence': 100,
 'Latitude': -36.7957,
 'Longitude': 143.9652,
 'Surface_Temperature_Celcius': 87,
 '_id': 1637}
{'Confidence': 96,
 'Latitude': -36.9498,
 'Longitude': 142.2012,
 'Surface_Temperature_Celcius': 78,
 '_id': 1640}
{'Confidence': 93,
 'Latitude': -36.1941,
 'Longitude': 1

 'Surface_Temperature_Celcius': 91,
 '_id': 1935}
{'Confidence': 100,
 'Latitude': -35.1857,
 'Longitude': 141.0604,
 'Surface_Temperature_Celcius': 92,
 '_id': 1938}
{'Confidence': 100,
 'Latitude': -35.6469,
 'Longitude': 142.376,
 'Surface_Temperature_Celcius': 93,
 '_id': 1947}
{'Confidence': 91,
 'Latitude': -35.6453,
 'Longitude': 142.387,
 'Surface_Temperature_Celcius': 69,
 '_id': 1948}
{'Confidence': 86,
 'Latitude': -35.9124,
 'Longitude': 141.8639,
 'Surface_Temperature_Celcius': 61,
 '_id': 1951}
{'Confidence': 100,
 'Latitude': -36.4409,
 'Longitude': 141.6948,
 'Surface_Temperature_Celcius': 92,
 '_id': 1953}
{'Confidence': 100,
 'Latitude': -37.9174,
 'Longitude': 145.8678,
 'Surface_Temperature_Celcius': 93,
 '_id': 1956}
{'Confidence': 95,
 'Latitude': -37.9152,
 'Longitude': 145.8808,
 'Surface_Temperature_Celcius': 77,
 '_id': 1957}
{'Confidence': 95,
 'Latitude': -37.8906,
 'Longitude': 146.2035,
 'Surface_Temperature_Celcius': 75,
 '_id': 1960}
{'Confidence': 86,
 

{'Confidence': 82,
 'Latitude': -36.8395,
 'Longitude': 142.2188,
 'Surface_Temperature_Celcius': 62,
 '_id': 2370}
{'Confidence': 97,
 'Latitude': -36.1925,
 'Longitude': 145.93,
 'Surface_Temperature_Celcius': 79,
 '_id': 2373}
{'Confidence': 86,
 'Latitude': -37.8351,
 'Longitude': 145.9789,
 'Surface_Temperature_Celcius': 61,
 '_id': 2374}
{'Confidence': 98,
 'Latitude': -36.095,
 'Longitude': 141.7699,
 'Surface_Temperature_Celcius': 82,
 '_id': 2375}
{'Confidence': 85,
 'Latitude': -36.7848,
 'Longitude': 141.9044,
 'Surface_Temperature_Celcius': 61,
 '_id': 2376}
{'Confidence': 96,
 'Latitude': -36.6006,
 'Longitude': 141.9657,
 'Surface_Temperature_Celcius': 78,
 '_id': 2385}
{'Confidence': 93,
 'Latitude': -37.0769,
 'Longitude': 141.042,
 'Surface_Temperature_Celcius': 72,
 '_id': 2387}
{'Confidence': 79,
 'Latitude': -36.883,
 'Longitude': 142.1637,
 'Surface_Temperature_Celcius': 67,
 '_id': 2390}
{'Confidence': 87,
 'Latitude': -36.0973,
 'Longitude': 143.4279,
 'Surface_T

#### 3. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2017

In [15]:
results = db.climate_historic.aggregate([
{
    "$lookup": {
        "from": "hotspot_historic",
        "localField": "Date",
        "foreignField" : "Date",
        "as": "hotspot_data"
    }
}, 
{
    "$match": {
        "$or": [ 
            {"Date": "2017-12-15"}, 
            {"Date": "2017-12-16"}
        ]
    }
},
{
    "$project": {
        "Date": 1,
        "hotspot_data.Surface_Temperature_Celcius": 1,
        "Air_Temperature_Celcius": 1,
        "Relative_Humidity": 1,
        "Max_Wind_Speed": 1
    }
}])
for result in results:
    pprint(result)

{'Air_Temperature_Celcius': 18,
 'Date': '2017-12-15',
 'Max_Wind_Speed': 14.0,
 'Relative_Humidity': 52.0,
 '_id': 348,
 'hotspot_data': [{'Surface_Temperature_Celcius': 42},
                  {'Surface_Temperature_Celcius': 36},
                  {'Surface_Temperature_Celcius': 38},
                  {'Surface_Temperature_Celcius': 40}]}
{'Air_Temperature_Celcius': 18,
 'Date': '2017-12-16',
 'Max_Wind_Speed': 13.0,
 'Relative_Humidity': 53.7,
 '_id': 349,
 'hotspot_data': [{'Surface_Temperature_Celcius': 43},
                  {'Surface_Temperature_Celcius': 33},
                  {'Surface_Temperature_Celcius': 54},
                  {'Surface_Temperature_Celcius': 73},
                  {'Surface_Temperature_Celcius': 55},
                  {'Surface_Temperature_Celcius': 75},
                  {'Surface_Temperature_Celcius': 55},
                  {'Surface_Temperature_Celcius': 66},
                  {'Surface_Temperature_Celcius': 56},
                  {'Surface_Temperature_Ce

#### 4. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100

In [16]:
results = db.hotspot_historic.aggregate([
{
    "$match": {
        "Confidence": { 
            "$gte": 80,
            "$lte": 100
        }
    }
},
{
    "$lookup": {
        "from": "climate_historic",
        "localField": "Date",
        "foreignField": "Date",
        "as": "climate_data"
    }
},
{
    "$project": {
        "Datetime": 1,
        "Surface_Temperature_Celcius": 1,
        "Air_Temperature_Celcius": "$climate_data.Air_Temperature_Celcius",
        "Confidence": 1
    }
},
])
for result in results:
    pprint(result)

{'Air_Temperature_Celcius': [28],
 'Confidence': 82,
 'Datetime': datetime.datetime(2017, 12, 27, 0, 2, 15),
 'Surface_Temperature_Celcius': 63,
 '_id': 1}
{'Air_Temperature_Celcius': [28],
 'Confidence': 86,
 'Datetime': datetime.datetime(2017, 12, 27, 0, 2, 14),
 'Surface_Temperature_Celcius': 67,
 '_id': 3}
{'Air_Temperature_Celcius': [17],
 'Confidence': 80,
 'Datetime': datetime.datetime(2017, 12, 25, 4, 29, 8),
 'Surface_Temperature_Celcius': 54,
 '_id': 4}
{'Air_Temperature_Celcius': [18],
 'Confidence': 94,
 'Datetime': datetime.datetime(2017, 12, 16, 15, 38, 39),
 'Surface_Temperature_Celcius': 43,
 '_id': 7}
{'Air_Temperature_Celcius': [18],
 'Confidence': 93,
 'Datetime': datetime.datetime(2017, 12, 16, 4, 35, 13),
 'Surface_Temperature_Celcius': 73,
 '_id': 10}
{'Air_Temperature_Celcius': [18],
 'Confidence': 84,
 'Datetime': datetime.datetime(2017, 12, 16, 4, 34, 58),
 'Surface_Temperature_Celcius': 55,
 '_id': 11}
{'Air_Temperature_Celcius': [18],
 'Confidence': 95,
 'Dat

{'Air_Temperature_Celcius': [14],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 9, 24, 15, 7, 47),
 'Surface_Temperature_Celcius': 65,
 '_id': 213}
{'Air_Temperature_Celcius': [14],
 'Confidence': 90,
 'Datetime': datetime.datetime(2017, 9, 24, 15, 7, 47),
 'Surface_Temperature_Celcius': 41,
 '_id': 216}
{'Air_Temperature_Celcius': [14],
 'Confidence': 99,
 'Datetime': datetime.datetime(2017, 9, 24, 15, 7, 47),
 'Surface_Temperature_Celcius': 61,
 '_id': 217}
{'Air_Temperature_Celcius': [14],
 'Confidence': 94,
 'Datetime': datetime.datetime(2017, 9, 24, 15, 7, 45),
 'Surface_Temperature_Celcius': 43,
 '_id': 220}
{'Air_Temperature_Celcius': [14],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 9, 24, 15, 7, 45),
 'Surface_Temperature_Celcius': 61,
 '_id': 221}
{'Air_Temperature_Celcius': [14],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 9, 24, 13, 30, 10),
 'Surface_Temperature_Celcius': 47,
 '_id': 222}
{'Air_Temperature_Celcius': [14],
 'Confidence':

 'Surface_Temperature_Celcius': 63,
 '_id': 555}
{'Air_Temperature_Celcius': [9],
 'Confidence': 85,
 'Datetime': datetime.datetime(2017, 5, 13, 4, 39),
 'Surface_Temperature_Celcius': 60,
 '_id': 558}
{'Air_Temperature_Celcius': [9],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 5, 13, 4, 38, 50),
 'Surface_Temperature_Celcius': 93,
 '_id': 560}
{'Air_Temperature_Celcius': [9],
 'Confidence': 87,
 'Datetime': datetime.datetime(2017, 5, 13, 4, 38, 40),
 'Surface_Temperature_Celcius': 62,
 '_id': 562}
{'Air_Temperature_Celcius': [9],
 'Confidence': 81,
 'Datetime': datetime.datetime(2017, 5, 13, 4, 38, 40),
 'Surface_Temperature_Celcius': 54,
 '_id': 564}
{'Air_Temperature_Celcius': [9],
 'Confidence': 86,
 'Datetime': datetime.datetime(2017, 5, 13, 4, 38, 40),
 'Surface_Temperature_Celcius': 60,
 '_id': 567}
{'Air_Temperature_Celcius': [9],
 'Confidence': 84,
 'Datetime': datetime.datetime(2017, 5, 13, 4, 38, 40),
 'Surface_Temperature_Celcius': 57,
 '_id': 568}
{'Air_Temper

 'Surface_Temperature_Celcius': 59,
 '_id': 786}
{'Air_Temperature_Celcius': [12],
 'Confidence': 98,
 'Datetime': datetime.datetime(2017, 5, 6, 4, 32, 20),
 'Surface_Temperature_Celcius': 82,
 '_id': 787}
{'Air_Temperature_Celcius': [12],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 5, 6, 4, 32, 20),
 'Surface_Temperature_Celcius': 90,
 '_id': 789}
{'Air_Temperature_Celcius': [12],
 'Confidence': 83,
 'Datetime': datetime.datetime(2017, 5, 6, 0, 16, 40),
 'Surface_Temperature_Celcius': 57,
 '_id': 792}
{'Air_Temperature_Celcius': [12],
 'Confidence': 92,
 'Datetime': datetime.datetime(2017, 5, 6, 0, 13, 50),
 'Surface_Temperature_Celcius': 71,
 '_id': 793}
{'Air_Temperature_Celcius': [14],
 'Confidence': 95,
 'Datetime': datetime.datetime(2017, 5, 5, 3, 59, 10),
 'Surface_Temperature_Celcius': 77,
 '_id': 795}
{'Air_Temperature_Celcius': [14],
 'Confidence': 81,
 'Datetime': datetime.datetime(2017, 5, 5, 3, 57),
 'Surface_Temperature_Celcius': 54,
 '_id': 796}
{'Air_Temper

 'Surface_Temperature_Celcius': 90,
 '_id': 1055}
{'Air_Temperature_Celcius': [14],
 'Confidence': 82,
 'Datetime': datetime.datetime(2017, 4, 25, 5, 2, 30),
 'Surface_Temperature_Celcius': 56,
 '_id': 1059}
{'Air_Temperature_Celcius': [19],
 'Confidence': 86,
 'Datetime': datetime.datetime(2017, 4, 24, 4, 8, 20),
 'Surface_Temperature_Celcius': 78,
 '_id': 1066}
{'Air_Temperature_Celcius': [19],
 'Confidence': 94,
 'Datetime': datetime.datetime(2017, 4, 24, 4, 8, 20),
 'Surface_Temperature_Celcius': 108,
 '_id': 1068}
{'Air_Temperature_Celcius': [17],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 23, 12, 55),
 'Surface_Temperature_Celcius': 50,
 '_id': 1071}
{'Air_Temperature_Celcius': [17],
 'Confidence': 91,
 'Datetime': datetime.datetime(2017, 4, 23, 12, 50, 50),
 'Surface_Temperature_Celcius': 43,
 '_id': 1072}
{'Air_Temperature_Celcius': [17],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 23, 12, 50, 40),
 'Surface_Temperature_Celcius': 55,
 '_id': 107

...

 '_id': 1591}
{'Air_Temperature_Celcius': [12],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 15, 4, 14, 30),
 'Surface_Temperature_Celcius': 116,
 '_id': 1593}
{'Air_Temperature_Celcius': [12],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 15, 4, 14, 30),
 'Surface_Temperature_Celcius': 111,
 '_id': 1595}
{'Air_Temperature_Celcius': [12],
 'Confidence': 96,
 'Datetime': datetime.datetime(2017, 4, 15, 4, 14, 20),
 'Surface_Temperature_Celcius': 79,
 '_id': 1597}
{'Air_Temperature_Celcius': [12],
 'Confidence': 81,
 'Datetime': datetime.datetime(2017, 4, 15, 4, 14, 20),
 'Surface_Temperature_Celcius': 55,
 '_id': 1598}
{'Air_Temperature_Celcius': [12],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 15, 4, 14, 20),
 'Surface_Temperature_Celcius': 90,
 '_id': 1600}
{'Air_Temperature_Celcius': [12],
 'Confidence': 93,
 'Datetime': datetime.datetime(2017, 4, 15, 4, 14, 20),
 'Surface_Temperature_Celcius': 73,
 '_id': 1601}
{'Air_Temperature_Celcius':

 'Datetime': datetime.datetime(2017, 4, 13, 4, 26, 50),
 'Surface_Temperature_Celcius': 59,
 '_id': 1795}
{'Air_Temperature_Celcius': [16],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 13, 4, 26, 40),
 'Surface_Temperature_Celcius': 104,
 '_id': 1797}
{'Air_Temperature_Celcius': [16],
 'Confidence': 90,
 'Datetime': datetime.datetime(2017, 4, 13, 4, 26, 40),
 'Surface_Temperature_Celcius': 66,
 '_id': 1808}
{'Air_Temperature_Celcius': [16],
 'Confidence': 80,
 'Datetime': datetime.datetime(2017, 4, 13, 4, 26, 40),
 'Surface_Temperature_Celcius': 53,
 '_id': 1810}
{'Air_Temperature_Celcius': [16],
 'Confidence': 97,
 'Datetime': datetime.datetime(2017, 4, 13, 4, 26, 40),
 'Surface_Temperature_Celcius': 80,
 '_id': 1811}
{'Air_Temperature_Celcius': [16],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 13, 4, 26, 40),
 'Surface_Temperature_Celcius': 97,
 '_id': 1812}
{'Air_Temperature_Celcius': [16],
 'Confidence': 90,
 'Datetime': datetime.datetime(2017, 4, 13,

 '_id': 2114}
{'Air_Temperature_Celcius': [21],
 'Confidence': 86,
 'Datetime': datetime.datetime(2017, 4, 7, 12, 53, 40),
 'Surface_Temperature_Celcius': 41,
 '_id': 2117}
{'Air_Temperature_Celcius': [21],
 'Confidence': 86,
 'Datetime': datetime.datetime(2017, 4, 7, 12, 52, 10),
 'Surface_Temperature_Celcius': 41,
 '_id': 2119}
{'Air_Temperature_Celcius': [21],
 'Confidence': 97,
 'Datetime': datetime.datetime(2017, 4, 7, 12, 51, 20),
 'Surface_Temperature_Celcius': 45,
 '_id': 2120}
{'Air_Temperature_Celcius': [21],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 7, 12, 51),
 'Surface_Temperature_Celcius': 49,
 '_id': 2123}
{'Air_Temperature_Celcius': [21],
 'Confidence': 92,
 'Datetime': datetime.datetime(2017, 4, 7, 12, 50, 50),
 'Surface_Temperature_Celcius': 43,
 '_id': 2124}
{'Air_Temperature_Celcius': [21],
 'Confidence': 89,
 'Datetime': datetime.datetime(2017, 4, 7, 12, 50, 40),
 'Surface_Temperature_Celcius': 42,
 '_id': 2130}
{'Air_Temperature_Celcius': [21],
 

 '_id': 2377}
{'Air_Temperature_Celcius': [16],
 'Confidence': 82,
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
 'Surface_Temperature_Celcius': 55,
 '_id': 2378}
{'Air_Temperature_Celcius': [16],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
 'Surface_Temperature_Celcius': 123,
 '_id': 2381}
{'Air_Temperature_Celcius': [16],
 'Confidence': 82,
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
 'Surface_Temperature_Celcius': 55,
 '_id': 2382}
{'Air_Temperature_Celcius': [16],
 'Confidence': 100,
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
 'Surface_Temperature_Celcius': 118,
 '_id': 2383}
{'Air_Temperature_Celcius': [16],
 'Confidence': 80,
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 40),
 'Surface_Temperature_Celcius': 54,
 '_id': 2384}
{'Air_Temperature_Celcius': [16],
 'Confidence': 96,
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 40),
 'Surface_Temperature_Celcius': 78,
 '_id': 2385}
{'Air_Temperature_Celcius': [16],


{'Air_Temperature_Celcius': [23],
 'Confidence': 82,
 'Datetime': datetime.datetime(2017, 3, 26, 4, 39, 10),
 'Surface_Temperature_Celcius': 55,
 '_id': 2577}
{'Air_Temperature_Celcius': [23],
 'Confidence': 87,
 'Datetime': datetime.datetime(2017, 3, 26, 4, 39, 10),
 'Surface_Temperature_Celcius': 62,
 '_id': 2578}
{'Air_Temperature_Celcius': [23],
 'Confidence': 84,
 'Datetime': datetime.datetime(2017, 3, 26, 4, 39, 10),
 'Surface_Temperature_Celcius': 58,
 '_id': 2580}
{'Air_Temperature_Celcius': [23],
 'Confidence': 82,
 'Datetime': datetime.datetime(2017, 3, 25, 3, 59, 50),
 'Surface_Temperature_Celcius': 65,
 '_id': 2586}
{'Air_Temperature_Celcius': [23],
 'Confidence': 87,
 'Datetime': datetime.datetime(2017, 3, 25, 3, 58, 10),
 'Surface_Temperature_Celcius': 62,
 '_id': 2587}
{'Air_Temperature_Celcius': [23],
 'Confidence': 95,
 'Datetime': datetime.datetime(2017, 3, 25, 3, 58, 10),
 'Surface_Temperature_Celcius': 75,
 '_id': 2588}
{'Air_Temperature_Celcius': [23],
 'Confidence

#### 5. Find the top 10 records with the highest surface temperature (°C)

In [17]:
results = db.hotspot_historic.aggregate([
{
    "$sort" : { 
        "Surface_Temperature_Celcius" : -1
    }
},
{
    "$limit" : 10
}])
for result in results:
    pprint(result)

{'Confidence': 100,
 'Date': '2017-04-18',
 'Datetime': datetime.datetime(2017, 4, 18, 4, 52),
 'Latitude': -38.1665,
 'Longitude': 143.062,
 'Surface_Temperature_Celcius': 124,
 '_id': 1187}
{'Confidence': 100,
 'Date': '2017-04-04',
 'Datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
 'Latitude': -36.343,
 'Longitude': 142.1986,
 'Surface_Temperature_Celcius': 123,
 '_id': 2381}
{'Confidence': 100,
 'Date': '2017-05-01',
 'Datetime': datetime.datetime(2017, 5, 1, 4, 14, 20),
 'Latitude': -36.9318,
 'Longitude': 143.0907,
 'Surface_Temperature_Celcius': 122,
 '_id': 1045}
{'Confidence': 100,
 'Date': '2017-03-18',
 'Datetime': datetime.datetime(2017, 3, 18, 3, 50, 50),
 'Latitude': -37.016999999999996,
 'Longitude': 148.1297,
 'Surface_Temperature_Celcius': 121,
 '_id': 2620}
{'Confidence': 100,
 'Date': '2017-05-13',
 'Datetime': datetime.datetime(2017, 5, 13, 4, 40, 20),
 'Latitude': -34.9938,
 'Longitude': 141.876,
 'Surface_Temperature_Celcius': 120,
 '_id': 551}
{'Confidence':

#### 6. Find the number of fire in each day. You are required to only display the total number of fire and the date in the output

In [18]:
results = db.hotspot_historic.aggregate([
{
    "$group" : {
        '_id' : '$Date',
        'countfires' : {'$sum': 1}
    }
},

{ "$project" :
    {
        'Date':'$_id',
        '_id':0,
        'countfires':1
    }
}
])
for result in results:
    pprint(result)

{'Date': '2017-03-09', 'countfires': 3}
{'Date': '2017-03-10', 'countfires': 8}
{'Date': '2017-03-13', 'countfires': 2}
{'Date': '2017-03-15', 'countfires': 7}
{'Date': '2017-03-18', 'countfires': 3}
{'Date': '2017-03-19', 'countfires': 21}
{'Date': '2017-03-24', 'countfires': 2}
{'Date': '2017-03-28', 'countfires': 54}
{'Date': '2017-03-29', 'countfires': 1}
{'Date': '2017-04-02', 'countfires': 5}
{'Date': '2017-04-03', 'countfires': 72}
{'Date': '2017-04-05', 'countfires': 49}
{'Date': '2017-04-11', 'countfires': 24}
{'Date': '2017-04-12', 'countfires': 69}
{'Date': '2017-04-07', 'countfires': 39}
{'Date': '2017-04-13', 'countfires': 357}
{'Date': '2017-04-14', 'countfires': 18}
{'Date': '2017-04-15', 'countfires': 69}
{'Date': '2017-04-17', 'countfires': 38}
{'Date': '2017-04-18', 'countfires': 325}
{'Date': '2017-04-20', 'countfires': 31}
{'Date': '2017-04-22', 'countfires': 2}
{'Date': '2017-04-24', 'countfires': 8}
{'Date': '2017-05-01', 'countfires': 20}
{'Date': '2017-04-04', '

#### 7. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output

Aggregates on the data from climate and fire collections. Projects only the date and surface temperatures. Redacts the records which only have a surface temparature list size of more than 0 i.e. removes the empty lists. Averages the surface temperatures in the list into a single average surface temperature variable.

In [19]:
results = db.climate_historic.aggregate([
{
    "$lookup":{
        "from": "hotspot_historic",
        "localField": "Date",
        "foreignField": "Date",
        "as": "hotspot_data"
    }
},
{
    "$project": {
        "Date": 1,
        "Surface_Temperatures": "$hotspot_data.Surface_Temperature_Celcius"
    }
},
{
    "$redact": {
        "$cond": {
            "if": { "$gt" : [{"$size": "$Surface_Temperatures"}, 0]},
            "then": "$$DESCEND",
            "else": "$$PRUNE"
        }
    }
},
{
    "$project": {
        "Date": 1,
        "Average_Surface_Temperature": {"$avg": "$Surface_Temperatures"},
        "_id": 0
    }
}])
for result in results:
    pprint(result)

{'Average_Surface_Temperature': 60.5, 'Date': '2017-03-06'}
{'Average_Surface_Temperature': 64.0, 'Date': '2017-03-07'}
{'Average_Surface_Temperature': 51.5, 'Date': '2017-03-08'}
{'Average_Surface_Temperature': 46.666666666666664, 'Date': '2017-03-09'}
{'Average_Surface_Temperature': 69.375, 'Date': '2017-03-10'}
{'Average_Surface_Temperature': 88.2, 'Date': '2017-03-12'}
{'Average_Surface_Temperature': 38.5, 'Date': '2017-03-13'}
{'Average_Surface_Temperature': 65.6, 'Date': '2017-03-14'}
{'Average_Surface_Temperature': 46.0, 'Date': '2017-03-15'}
{'Average_Surface_Temperature': 59.5, 'Date': '2017-03-17'}
{'Average_Surface_Temperature': 79.33333333333333, 'Date': '2017-03-18'}
{'Average_Surface_Temperature': 65.57142857142857, 'Date': '2017-03-19'}
{'Average_Surface_Temperature': 49.0, 'Date': '2017-03-24'}
{'Average_Surface_Temperature': 66.0, 'Date': '2017-03-25'}
{'Average_Surface_Temperature': 56.88235294117647, 'Date': '2017-03-26'}
{'Average_Surface_Temperature': 60.9259259259